# Generating conditional probability tables subject to constraints


In [1]:
import os
from pathlib import Path

from itertools import product

import numpy as np
import pandas as pd

from fake_data_for_learning.fake_data_for_learning import (
    BayesianNodeRV, FakeDataBayesianNetwork, SampleValue
)
from fake_data_for_learning.utils import RandomCpt
from fake_data_for_learning.probability_polytopes import (
    MapMultidimIndexToLinear, ProbabilityPolytope, ExpectationConstraint
)

Suppose we want to generate data from a discrete Bayesian network, such as

Product -> Days <- Rating, 

where e.g. Product is the (insurance) product name, Rating is rating strength (i.e. market price / technical price) for a submission, and Days is the number of days to generate a quote for the submission.

The number of entries in probability and conditional probability tables to define this Bayesian network is

$ | Product | + | Rating | + | Product | \times | Rating | \times | Days |$.

For example, let us define Industry and Rating as follows

In [2]:
product_values = ['financial', 'liability', 'property']
product_type = BayesianNodeRV('product_type', np.array([0.2, 0.5, 0.3]), values=product_values)
rating_values = range(2)
rating = BayesianNodeRV('rating', np.array([0.3, 0.7]))

Suppose that Days is also discrete, e.g.

In [3]:
days_values = range(4)

Then if we choose the ordering of the conditional probability table axes as Product, Rating, Days, we can generate the entries of the conditional probability table for Days conditioned on Industry and Rating with `utils.RandomCpt`:

In [4]:
random_cpt = RandomCpt(len(product_values), len(rating_values), len(days_values))
X = random_cpt()

In [5]:
X[0, 0, :].sum()

0.9999999999999999

So the total number of probability table entries to specify is, as in the formula above,

In [6]:
f'Number of probability table entries: {len(product_values) + len(rating_values) +  (len(product_values) * len(rating_values) * len(days_values))}'

'Number of probability table entries: 29'

It would be nice to specify certain properties of the matrix without having to change entries individually. For example, we may want to insist that

\begin{equation*}
E(D | P = property) = 3.5 \\
E(D | P = financial) = 1.0 \\
E(D | P= liability) = 2.0
\end{equation*}

Denote the entries of the conditional probability table as 

$$(\rho_{p, r | d})$$

The the above constraints become

\begin{equation*}
\frac{1}{|R|} \sum_{r, d} d \, \rho_{\mathrm{property},\, r\, | d} = 3.5 \\
\frac{1}{|R|} \sum_{r, d} d \, \rho_{\mathrm{financial},\, r\, | d} = 1.0\\
\frac{1}{|R|} \sum_{r, d} d \, \rho_{\mathrm{liability},\, r\, | d} = 2.0.
\end{equation*}

As $(\rho)$ is a conditional probability table, we also have the constraints 

\begin{equation*}
0 \leq \rho_{p,\,r\,|d} \leq 1 \textrm{ for all }(p,\,r,\,d),\\
\sum_{d} \rho_{p,\,r,\,| d} = 1 \textrm{ for each pair } (p, \, r)
\end{equation*}

Together, these constraints define convex polytope contained in (probability) simplex $\Delta_{R-1} \subseteq \mathbb{R}^{R}$, where $R = |Product | \times | Rating | \times | Days|$ (see e.g. Chapter 1 of *Lectures on Algebraic Statistics*, Drton, Sturmfels, Sullivant). This polytope is defined as an intersection of half-spaces, i.e. using the so-called *H-representation* of the polytope, see *Lectures on Polytopes* by Ziegler, Chapters 0 and 1.

To generate a random (conditional) probability table to these constraints, the vertex-, or *V-representation* of the probability polytope $P$ is much more useful, because given the a vertex matrix $V$, where each column is a vertex of $P$ in $\mathbb{R}^R$, and all points in $P$ can be obtained as

$$
\begin{equation*}
x = V \cdot t
\end{equation*}
$$

where $t \in \mathbb{R}^N$, with $N$ being the number of vertices for $P$, and $t$ satisfying $0 \leq t_i \leq 1$, $\sum t_i = 1$.

Once we have determined the V-representation $V$, then the problem of generating conditional probability tables subject to our given expectation value constraints reduces to the much simpler problem of generating points on the non-negative quadrant of the unit (hyper) cube in $R^N$.

Before we get to our goal of generating these probability tables for our hit ratio problem, let's look at elementary examples.

## (Conditional) Probability Polytopes

The simplest example of a probability polytope is that of a Bernoulli random variable.

In [7]:
bernoulli = ProbabilityPolytope(('outcome',), dict(outcome=range(2)))
A, b = bernoulli.get_probability_half_planes()
print(A, '\n', b)

[[ 1.  1.]
 [-1. -1.]
 [ 1.  0.]
 [ 0.  1.]
 [-1. -0.]
 [-0. -1.]] 
 [ 1. -1.  1.  1.  0.  0.]


We convert the formulation A x <= b to the V-description

In [8]:
bernoulli.get_vertex_representation()

array([[1., 0.],
       [0., 1.]])

In [9]:
tertiary = ProbabilityPolytope(('outcome',), dict(outcome=range(3)))
tertiary.get_vertex_representation()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [10]:
conditional_bernoullis = ProbabilityPolytope(
    ('input', 'output'), dict(input=range(2), output=range(2))
)
conditional_bernoullis.get_vertex_representation()

array([[1., 1., 0., 0.],
       [0., 0., 1., 1.],
       [1., 0., 0., 1.],
       [0., 1., 1., 0.]])

The benefit of having the vertex-representation (V-representation) of the probability polytope is that generating random (conditional) probability tables is straightforward, namely, we can get all elements of the probability polytope by taking combinations of the vertex (column) vectors.

In the flattened coordinates, we have, e.g.

In [11]:
conditional_bernoullis.generate_flat_random_cpt()

array([0.27086489, 0.72913511, 0.74258253, 0.25741747])

In the multidimensional coordinates for conditional probability tables here, we have e.g.

In [12]:
conditional_bernoullis.generate_random_cpt()

array([[0.58372782, 0.41627218],
       [0.71527302, 0.28472698]])

## Adding contraints on conditional expectation values

In [13]:
conditional_bernoullis.set_expectation_constraints(
    [ExpectationConstraint(equation=dict(input=1), moment=1, value=0.5)]
)

In [14]:
conditional_bernoullis.get_expect_equations_col_indices(conditional_bernoullis.expect_constraints[0].equation)

[2, 3]

In [15]:
conditional_bernoullis.get_vertex_representation()

array([[1. , 0. ],
       [0. , 1. ],
       [0.5, 0.5],
       [0.5, 0.5]])

In [16]:
conditional_bernoullis.generate_random_cpt()

array([[0.76396714, 0.23603286],
       [0.5       , 0.5       ]])

In [17]:
two_input_constrained_polytope = ProbabilityPolytope(
    ('input', 'more_input', 'output'),
    dict(input=['hi', 'low'], more_input=range(2), output=range(2))
)
two_input_constrained_polytope.set_expectation_constraints(
    [ExpectationConstraint(equation=dict(more_input=0), moment=1, value=0.25)]
)
two_input_constrained_polytope.get_vertex_representation()

array([[0.5, 0.5, 0.5, 0.5, 1. , 1. , 1. , 1. ],
       [0.5, 0.5, 0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 1. , 1. , 1. , 0. , 0. ],
       [1. , 1. , 0. , 0. , 0. , 0. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 0.5, 0.5, 0.5, 0.5],
       [0. , 0. , 0. , 0. , 0.5, 0.5, 0.5, 0.5],
       [1. , 0. , 1. , 0. , 1. , 0. , 0. , 1. ],
       [0. , 1. , 0. , 1. , 0. , 1. , 1. , 0. ]])

## Hit rate polytope again

In [18]:
days_polytope = ProbabilityPolytope(
    ('product', 'rating', 'days'),
    coords = {
        'product': product_values, 
        'rating': rating_values, 
        'days': days_values
    }
)
days_polytope.set_expectation_constraints(
    [
        ExpectationConstraint(equation=dict(product='financial'), moment=1, value=0.2),
        ExpectationConstraint(equation=dict(product='liability'), moment=1, value=0.9),
        ExpectationConstraint(equation=dict(product='property'), moment=1, value=0.5),
    ]
)
days_cpt = days_polytope.generate_random_cpt()
days_cpt

array([[[0.88064391, 0.06169566, 0.03086844, 0.02679199],
        [0.87308745, 0.07564295, 0.03326026, 0.01800934]],

       [[0.40923928, 0.35176295, 0.14170279, 0.09729497],
        [0.44115411, 0.32982671, 0.14393771, 0.08508147]],

       [[0.66932198, 0.18715179, 0.08308226, 0.06044397],
        [0.71299054, 0.15674574, 0.08218511, 0.0480786 ]]])

Now we create our Bayesian network with desired constraints on some expectation values

In [19]:
days = BayesianNodeRV('days', days_cpt, parent_names=['product_type', 'rating'])
bn = FakeDataBayesianNetwork(product_type, rating)#, days)
bn = FakeDataBayesianNetwork(product_type, rating, days)
bn.rvs(10)

,product_type,rating,days
0,financial,1,1
1,property,1,0
2,financial,1,0
3,liability,1,0
4,property,1,0
5,property,0,1
6,liability,1,0
7,property,1,0
8,financial,0,0
9,liability,1,0
